### 데이터셋 (공원)
- 사용 열 : "공원명", "공원구분", "소재지도로명주소", "소재지지번주소", "위도", "경도", "공원면적(㎡)"


In [1]:
import pandas as pd
import numpy as np
import os

os.chdir("/Users/shinjh/Desktop/likelion/mid_project/dataset_life_infra")

In [ ]:
## 함수 init (결측 위경도 추가용)
import pandas as pd
import requests
import json
from tqdm import tqdm
from geopy.geocoders import Nominatim

# Kakao API로 좌표 검색 함수
def get_location_kakao(address_data, kakao_api_key):
    url = f'https://dapi.kakao.com/v2/local/search/address.json?query={address_data}'
    headers = {"Authorization": f"KakaoAK {kakao_api_key}"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        api_json = response.json()
        if api_json['documents']:
            address = api_json['documents'][0]['address']
            return float(address['y']), float(address['x'])  # 위도, 경도 반환
    return None, None

# Nominatim으로 좌표 검색 함수
def get_location_nominatim(address_data):
    geolocator = Nominatim(user_agent="cocoin")
    location = geolocator.geocode(address_data)
    if location:
        return location.latitude, location.longitude  # 위도, 경도 반환
    return None, None

# 데이터프레임 업데이트 함수
def update_coordinates(df, kakao_api_key):
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        if not pd.isna(row['위도']) and not pd.isna(row['경도']):
            continue

        # 주소 정보 우선순위: 소재지지번주소 -> 소재지도로명주소
        address = row['소재지지번주소'] if not pd.isna(row['소재지지번주소']) else row['소재지도로명주소']

        # Kakao API 호출
        lat, lon = get_location_kakao(address, kakao_api_key)
        if lat is None or lon is None:
            try:
                # Kakao API 실패 시 Nominatim 호출
                lat, lon = get_location_nominatim(address)
            except:
                print(f"can't get info : {address}")
                continue

        # 결과 저장|
        df.at[idx, '위도'] = lat
        df.at[idx, '경도'] = lon
        

# Kakao API Key 설정
kakao_api_key = "20953ebae869efd8abdc3657eba800b6"  # 카카오 API 키를 여기에 입력

In [5]:
data = pd.read_csv("/Users/shinjh/Desktop/likelion/mid_project/dataset_life_infra/도시공원정보현황.csv", encoding="utf-8")
data.head()

,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적(㎡),공원보유시설(운동시설),공원보유시설(유희시설),공원보유시설(편익시설),공원보유시설(교양시설),공원보유시설(기타시설),지정고시일,관리기관명,전화번호,데이터기준일자
0,호명호수공원,문화공원,경기도 가평군 청평면 상지로64번길 77,경기도 가평군 청평면 상천리 1407-2,37.765806,127.458729,241538.0,NaN,광장+산책로+지당+화계+폭포+휴게쉼터,화장실+나들센터,전시회랑+숲속공연장+야외교실,농특산물체험장,2007-06-11,경기도 가평군청,031-580-2517,2023-11-22
1,제13호 소공원,소공원,경기도 가평군 가평읍 석봉로191번길 37,경기도 가평군 가평읍 읍내리 1055,37.831447,127.505818,1487.3,NaN,NaN,NaN,NaN,도로+광장+쉼터+파고라+앉음벽+평의자+디딤석+녹지,2019-01-31,경기도 가평군청,031-580-2372,2023-11-22
2,제6호 문화공원,문화공원,NaN,경기도 가평군 가평읍 대곡리 144-14,NaN,NaN,1783.0,NaN,NaN,NaN,NaN,휴게마당,2016-12-26,경기도 가평군청,031-580-4989,2023-11-22
3,제12호 소공원,소공원,경기도 가평군 가평읍 석봉로191번길 37,경기도 가평군 가평읍 읍내리 1054,37.831447,127.505818,589.5,NaN,NaN,NaN,NaN,도로+광장+쉼터+파고라+앉음벽+평의자+디딤석+녹지,2019-01-31,경기도 가평군청,031-580-2372,2023-11-22
4,목동 근린공원,근린공원,경기도 가평군 북면 화악산로 45-16,경기도 가평군 북면 목동리 913-1,37.889301,127.549631,43113.0,테니스장+게이트볼장,어린이놀이터,전망대,전시장,야생화화원+조경녹지+휴게광장+도로+주차장+산책로,1974-12-26,경기도 가평군청,031-580-2373,2023-11-22


In [6]:
data_part_sort = data[["공원명", "공원구분", "소재지도로명주소", "소재지지번주소", "위도", "경도", "공원면적(㎡)"]]

In [52]:
ilsan = data_part_sort[
    data_part_sort["소재지도로명주소"].combine_first(data_part_sort["소재지지번주소"]).str.contains("일산", na=False)].reset_index(drop=True)

bundang = data_part_sort[
    data_part_sort["소재지도로명주소"].combine_first(data_part_sort["소재지지번주소"]).str.contains("성남시 분당", na=False)].reset_index(drop=True)


In [55]:
ilsan.head()

,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적(㎡)
0,오마,어린이공원,경기도 고양시 일산서구 주엽로 220-4,경기도 고양시 일산서구 주엽동 34번지,37.676064,126.764584,5700.0
1,식사(은행마을공원),근린공원,경기도 고양시 일산동구 은행마을로 80-4,경기도 고양시 일산동구 식사동 1475번지,37.669891,126.806486,10000.0
2,하늘장미(일산2-5),어린이공원,경기도 고양시 일산동구 하늘마을로 102-2,경기도 고양시 일산동구 중산동 1702번지,37.680478,126.784715,1500.0
3,안곡습지,수변공원,경기도 고양시 일산동구 하늘마을로 39-5,경기도 고양시 일산동구 중산동 1712번지,37.684019,126.784489,77829.0
4,개나리(일산2-4),어린이공원,경기도 고양시 일산동구 소개울2길 43,경기도 고양시 일산동구 중산동 1736번지,37.683828,126.790138,1885.0


In [56]:
bundang.head()

,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적(㎡)
0,판교제7호(송현)공원,근린공원,NaN,경기도 성남시 분당구 삼평동 717-2,37.409025,127.116940,39233.5
1,판교제10호(낙생대)공원,근린공원,경기도 성남시 분당구 판교역로 75,경기도 성남시 분당구 백현동 523,37.387522,127.107671,428698.0
2,미래공원,어린이공원,NaN,경기도 성남시 분당구 서현동 309,37.371048,127.136006,3509.7
3,양촌공원,어린이공원,NaN,경기도 성남시 분당구 야탑동 178,37.407056,127.147037,1533.2
4,판교제12호(숲안)공원,어린이공원,NaN,경기도 성남시 분당구 백현동 548,37.388994,127.115944,4275.3


In [51]:
ilsan[ilsan['위도'].isna()]

,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적(㎡)
17,탄현주상복합(숯고개향나무),어린이공원,NaN,경기도 고양시 일산서구 탄현동 173-3,NaN,NaN,4733.0
31,수변공원,근린공원,NaN,경기도 고양시 일산동구 장항동 1097일원(1785-1),NaN,NaN,81511.0
35,백마,근린공원,NaN,경기도 고양시 일산동구 마두동 111-2(725),NaN,NaN,20490.0


In [ ]:
bundang[bundang['위도'].isna()]

,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적(㎡)
1241,남서울묘지공원,묘지공원,NaN,경기도 성남시 분당구 야탑동 산 82,NaN,NaN,634921.0


In [53]:
# 결측치 부분에 대해선는 수동으로 표기 (갯수가 적기에, 구글맵 활용하여 진행)
ilsan.to_csv("./dataset_life_infra_일산_공원.csv", encoding='utf-8')
bundang.to_csv("./dataset_life_infra_분당_공원.csv", encoding='utf-8')